In [ ]:
import os
import csv
import cv2
from skimage import io
import numpy as np
from skimage.measure import block_reduce, label, regionprops
from skimage.color import label2rgb
import tifffile as tiff
import matplotlib.pyplot as plt
from nd2 import ND2File
from pathlib import Path

from scipy import stats


import pandas as pd


In [ ]:
directory = Path('.')

# Get all .nd2 files
csv_files = list(directory.glob("*.csv"))

# If you want full paths as strings:
csv_file_paths = [str(f) for f in csv_files]

In [ ]:
csv_file_paths

In [ ]:
dfs = []

for csv in csv_file_paths:
    df = pd.read_csv(csv)
    dfs.append(df)

In [ ]:
(2+5)/(0.4315)


In [ ]:
for i, df in enumerate(dfs):

    print(f"Analyzing file {csv_file_paths[i]}")

    df_subset = df[df['distance'] <= 16]

    for j, comp_df in enumerate(dfs):

        comp_subset = comp_df[comp_df['distance'] <= 16]

        x = df_subset['distance']
        y = comp_subset['distance']
        stat, pvalue = (stats.kruskal(x,y))
        if pvalue < 0.05/(16*16):
            print(pvalue)
        else:
            pass

    print()
    


In [ ]:

x = dfs[0]['distance']
y = dfs[1]['distance']

stats.kruskal(x, y)

In [ ]:
dfs[0].head()

In [ ]:
all_prop_df = pd.read_csv("/Users/nelsschimek/Downloads/All_Properties.csv")
all_prop_df.head()

In [ ]:
all_prop_df["ideal_radius"].hist(bins=10, range=[0,20])